In [41]:
import numpy as np
from multiprocessing import Pool
import time

In [46]:
def arr2oct(arr):
    length = len(arr)
    temp = 8**np.arange(length)[::-1]
    print(np.sum(arr*temp))
    print(f"0o{np.sum(arr*temp):o}")
    return oct(np.sum(arr*temp))

In [47]:
import halo_defs as H
def pos2oct(pos):
    from num_rec import icellid, icellids
    sizeroot = 1
    nlevelmax = 6
    octarr = np.zeros((pos.shape[0], nlevelmax), dtype=np.int8)
    cpos = np.zeros((pos.shape[0],3), dtype=np.float64)
    for ilevel in range(nlevelmax):
        icids = icellids(pos - cpos)
        octarr[:,ilevel] = icids
        print(ilevel, octarr[:5,ilevel])
        # cpos += H.sizeroot*H.pos_ref_0[:3,icids] * 2**(-ilevel-2)
        cpos += sizeroot*H.pos_ref_0[:3,icids].T * 2**(-ilevel-2)
    temp = 8**np.arange(nlevelmax)[::-1]
    return np.sum(octarr*temp, axis=1)

In [54]:
from rur import uri
repo = '/storage3/Clusters/07206'
snap = uri.RamsesSnapshot(repo, 187, mode='yzics')
dm = snap.get_part(pname='star', nthread=32)

[Output 00187] Age (Gyr) : 13.741 / 13.741, z = -0.00000 (a = 1.0000)
Reading 768 part files (2.136 GiB) in /storage3/Clusters/07206/snapshots/output_00187... 
Done (1.768s).


In [55]:
pos = dm['pos']-np.array([0.5,0.5,0.5])
nlevelmax = 30


[(0, 0, 0, False, 0, 0) (0, 0, 0, False, 0, 0) (0, 0, 0, False, 0, 0) ...
 (0, 0, 0, False, 0, 0) (0, 0, 0, False, 0, 0) (0, 0, 0, False, 0, 0)]


In [56]:
kdtree = {}
ncellmx = 2*pos.shape[0]+1
kdtree['mass_cell'] = np.zeros(ncellmx, dtype=np.int64)
kdtree['pos_cell'] = np.zeros((ncellmx,3), dtype=np.float64)
kdtree['size_cell'] = np.zeros(ncellmx, dtype=np.int64)
kdtree['sister'] = np.zeros(ncellmx, dtype=np.int64)
kdtree['firstchild'] = np.zeros(ncellmx, dtype=np.int64)

In [64]:
def refine(pos, octarr, cpos, base=8):
    from num_rec import icellid, icellids
    sizeroot=1
    yet = ~octarr["done"]
    ncell_new = 1
    while(True in yet):
        nlevel = int( np.max(octarr["lvl"]) )
        octid = octarr['oct1'] if(nlevel < 15) else octarr['oct1']+1j*octarr['oct2']
        ncell_old = ncell_new
        icids = icellids(pos[yet]-cpos[yet])+1
        if(nlevel > 14):
            octarr['oct2'] *= base
            octarr['oct2'][yet] += icids
        else:
            octarr['oct1'] *= base
            octarr['oct1'][yet] += icids
        octarr['lvl'][yet] += 1
        if(nlevel >= 30): break

        octid = octarr['oct1'] if(nlevel < 15) else octarr['oct1']+1j*octarr['oct2']
        uni, count = np.unique(octid[yet], return_counts=True)
        if(True in (count<=20)):
            leafind = np.where(count<=20)[0]
            leaf = uni[leafind]
            isin = np.isin(octid, leaf)
            octarr["done"][isin] = True
        a,b = np.unique(octarr['lvl'], return_counts=True)
        ncell_new = np.sum(b[:-1]) + len( np.unique(octid[octarr['lvl'] == a[-1]]) )
        print(f"[lvl={nlevel}] {ncell_old} -> {ncell_new} [{octarr[0]}, {octarr[yet][0]}]")
        cpos[yet] += sizeroot*H.pos_ref_0[:3,icids-1].T * 2**(-nlevel-2)
        yet = ~octarr["done"]
    print(f"[lvl={nlevel}] KDtree done")
    return octarr

In [65]:
cpos = np.zeros((pos.shape[0],3), dtype=np.float128)
octarr = np.zeros(pos.shape[0],dtype=[
            ("oct1", np.int64),("oct2", np.int64), ("lvl", np.int8), ("done", bool), 
            ("inccell", np.int64), ("idmother", np.int64)
            ])
octarr["done"] = False
print(octarr)

[(0, 0, 0, False, 0, 0) (0, 0, 0, False, 0, 0) (0, 0, 0, False, 0, 0) ...
 (0, 0, 0, False, 0, 0) (0, 0, 0, False, 0, 0) (0, 0, 0, False, 0, 0)]


In [66]:
octarr = refine(pos, octarr, cpos, base=10)

[lvl=0] 1 -> 2 [(1, 0, 1, False, 0, 0), (1, 0, 1, False, 0, 0)]
[lvl=1] 2 -> 4 [(18, 0, 2, True, 0, 0), (18, 0, 2, True, 0, 0)]


KeyboardInterrupt: 

In [63]:
oct(1), oct(16), oct(128), oct(8247)

('0o1', '0o20', '0o200', '0o20067')

In [ ]:
        # mem['pos_cell'][:3,inccell-1]=pos_this_node[:3]
        # mem['mass_cell'][inccell-1]=npart_this_node
        # mem['size_cell'][inccell-1]=2.**(-nlevel)*H.sizeroot*0.5
        # if (idmother>0):
        #     # If this is not the root cell, we link it to its mother
        #     mem['sister'][inccell-1]=mem['firstchild'][idmother-1]
        #     mem['firstchild'][idmother-1]=inccell
        # if ((npart_this_node <= H.npartpercell) or (nlevel==H.nlevelmax)):
        #     # If there is only one particle in the H.node_0 or we have reach
        #     # maximum level of refinement, we are done
        #     mem['firstchild'][inccell-1]=-(first_pos_this_node+1)

In [341]:
lexsort = np.lexsort((octarr['oct2'], octarr['oct1']))
octarr[lexsort]

array([(111111100000000, 0, 7,  True, 0, 0),
       (111111100000000, 0, 7,  True, 0, 0),
       (111111100000000, 0, 7,  True, 0, 0), ...,
       (888888800000000, 0, 7,  True, 0, 0),
       (888888800000000, 0, 7,  True, 0, 0),
       (888888800000000, 0, 7,  True, 0, 0)],
      dtype=[('oct1', '<i8'), ('oct2', '<i8'), ('lvl', 'i1'), ('done', '?'), ('inccell', '<i8'), ('idmother', '<i8')])

In [ ]:
f"{}"

In [447]:
ids = np.arange(octarr.shape[0])+1
ids = ids[lexsort]

In [389]:
octree = octarr[lexsort]

In [464]:
def bign(ilevel):
    return int( 10**(15-ilevel) )

In [501]:
octree.shape

(11134418,)

In [503]:
octids = [f"{i['oct1']}{i['oct2']}" for i in octree]

In [511]:
octids = np.array(octids)

In [512]:
octids

array(['1111111000000000', '1111111000000000', '1111111000000000', ...,
       '8888888000000000', '8888888000000000', '8888888000000000'],
      dtype='<U19')

In [506]:
np.array([i[0] for i in octids], dtype=int)

array([1, 1, 1, ..., 8, 8, 8])

In [517]:
mass_cell = np.zeros(ncellmx, dtype=np.int64)-1
inccell=0
ilevel = 0
mass_cell[inccell] = len(octree); inccell+=1
other = octids

def gogo(ilevel, other):
    global inccell, mass_cell
    ilevel += 1
    n = np.array([i[0] for i in other], dtype=np.int8)
    for j in range(8):
        ind = n == j+1
        if(True in ind):
            mass_cell[inccell] = np.sum(ind); inccell+=1
            if((inccell%100000)==0): print('inccell=',inccell)
            # print(ilevel, j+1, mass_cell[inccell-1])
            temp = np.array([i[1:] for i in other[ind]])
            if(temp[0]==0): continue
            gogo(ilevel, temp)

gogo(ilevel, other)

# ilevel += 1
# N = bign(ilevel)
# n = other//N
# mass_cell[inccell] = np.sum(n==1); inccell+=1
# other = other[n==1]%N
# print(n[0],other[0])

# ilevel += 1
# N = bign(ilevel)
# n = other//N
# mass_cell[inccell] = np.sum(n==1); inccell+=1
# other = other[n==1]%N
# print(n[0],other[0])

# ilevel += 1
# N = bign(ilevel)
# n = other//N
# mass_cell[inccell] = np.sum(n==1); inccell+=1
# other = other[n==1]%N
# print(n[0],other[0])

# ilevel += 1
# N = bign(ilevel)
# n = other//N
# mass_cell[inccell] = np.sum(n==1); inccell+=1
# other = other[n==1]%N
# print(n[0],other[0])

# ilevel += 1
# N = bign(ilevel)
# n = other//N
# mass_cell[inccell] = np.sum(n==1); inccell+=1
# other = other[n==1]%N
# print(n[0],other[0])

# ilevel += 1
# N = bign(ilevel)
# n = other//N
# mass_cell[inccell] = np.sum(n==1); inccell+=1
# other = other[n==1]%N
# print(n[0],other[0])

# ilevel += 1
# N = bign(ilevel)
# n = other//N
# mass_cell[inccell] = np.sum(n==1); inccell+=1
# other = other[n==1]%N
# print(n[0],other[0])





mass_cell[:10]

inccell= 100000
inccell= 200000
inccell= 300000
inccell= 400000


KeyboardInterrupt: 

In [518]:
mass_cell[:30]

array([11134418,  4401030,    30283,     7327,      510,       81,
             27,        3,       14,        4,        1,        3,
              2,        7,        5,       16,        5,        3,
              8,       10,       31,        5,        5,        2,
              2,        5,        7,        3,        2,       81])

In [390]:
octree

array([(111111100000000, 0, 7,  True, 0, 0),
       (111111100000000, 0, 7,  True, 0, 0),
       (111111100000000, 0, 7,  True, 0, 0), ...,
       (888888800000000, 0, 7,  True, 0, 0),
       (888888800000000, 0, 7,  True, 0, 0),
       (888888800000000, 0, 7,  True, 0, 0)],
      dtype=[('oct1', '<i8'), ('oct2', '<i8'), ('lvl', 'i1'), ('done', '?'), ('inccell', '<i8'), ('idmother', '<i8')])

In [388]:
octarr

array([(111323000000000, 0, 6,  True, 0, 0),
       (111324300000000, 0, 7,  True, 0, 0),
       (111525780000000, 0, 8,  True, 0, 0), ...,
       (241888000000000, 0, 6,  True, 0, 0),
       (241888000000000, 0, 6,  True, 0, 0),
       (241888000000000, 0, 6,  True, 0, 0)],
      dtype=[('oct1', '<i8'), ('oct2', '<i8'), ('lvl', 'i1'), ('done', '?'), ('inccell', '<i8'), ('idmother', '<i8')])

In [360]:
int

int

In [346]:
np.log10(octarr['oct1']).astype(int)

array([14, 14, 14, ..., 14, 14, 14])

In [356]:
octarr['oct1'] // 1e14

array([1., 1., 1., ..., 2., 2., 2.])

In [358]:
octarr[0]['oct1'] / 1e14

1.11323

In [367]:
kdtree['mass_cell'] = []

In [384]:
kdtree['mass_cell'] = []
kdtree['mass_cell'].append(octarr.shape[0])
kdtree['mass_cell'].append([-1]*8)
nlevelmax = np.max(octarr['lvl'])
temparr = kdtree['mass_cell']
for ilevel in range(1,nlevelmax+1):
    if(ilevel>2): break
    print(temparr)
    temparr = temparr[1]
    for i in range(8):
        tmp = octarr['oct1'] // 1e14
        temparr[i] = [np.sum(tmp == (i+1)), [-1]*8]

[11134418, [-1, -1, -1, -1, -1, -1, -1, -1]]
[[4401030, [-1, -1, -1, -1, -1, -1, -1, -1]], [269003, [-1, -1, -1, -1, -1, -1, -1, -1]], [240826, [-1, -1, -1, -1, -1, -1, -1, -1]], [264320, [-1, -1, -1, -1, -1, -1, -1, -1]], [5175286, [-1, -1, -1, -1, -1, -1, -1, -1]], [242939, [-1, -1, -1, -1, -1, -1, -1, -1]], [260857, [-1, -1, -1, -1, -1, -1, -1, -1]], [280157, [-1, -1, -1, -1, -1, -1, -1, -1]]]


IndexError: list assignment index out of range

In [386]:
ilevel = 1
myoct = 1
tmp = octarr['oct1'] // int( 10**(15-ilevel) )
mychild = octarr['oct1'][tmp == myoct] - myoct*int( 10**(15-ilevel) )

for j in range(1,8+1):
    tmp = mychild // int( 10**(15-ilevel-1) )
    jthchild = mychild[tmp == j] - j*int( 10**(15-ilevel-1) )

In [393]:
octree[-1]['oct1'] // 1e15

0.0

In [439]:
np.max(octree['lvl'])

19

In [453]:
kdtree['mass_cell'] = [None] * 10000000

inccell = 1
class node:
    def __init__(self, lvl, j,mine, idlists):
        global inccell, kdtree
        self.inccell = inccell; inccell += 1
        if(inccell%100000 ==0): print(inccell)
        # if(inccell > 30): raise ValueError("stop")
        self.lvl = lvl
        self.j = j
        self.mine = mine
        self.ids = idlists
        self.npart = len(self.mine)
        # print(self.inccell, lvl, j, self.npart)
        self.childs = [None]*8
        self.mother = None
        self.done = False
        kdtree['mass_cell'][self.inccell-1] = self.npart
        self.init_tree()
    
    def init_tree(self):
        if(self.npart <= 20):
            self.done = True
            return
        
        global octree
        mine = self.mine%(10**(15-self.lvl))
        for j in range(8):
            ind = mine//(10**(15-self.lvl-1)) == j+1
            jmine = self.mine[ind]
            jids = self.ids[ind]
            child = node(self.lvl+1, j+1, jmine, jids)
            child.mother = self
            self.childs[j] = child

root = node(0, 0, octree['oct1'], ids)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000


In [456]:
np.max(octree['lvl'])

19

In [455]:
len(np.unique(octree['oct1']))

1909031

In [454]:
kdtree['mass_cell']

[11134418,
 4401030,
 30283,
 7327,
 510,
 81,
 27,
 3,
 14,
 0,
 4,
 1,
 3,
 0,
 2,
 7,
 5,
 16,
 5,
 3,
 8,
 10,
 31,
 5,
 5,
 2,
 2,
 5,
 7,
 3,
 2,
 81,
 4,
 22,
 2,
 3,
 0,
 4,
 2,
 3,
 2,
 6,
 4,
 12,
 5,
 20,
 1,
 13,
 58,
 6,
 2,
 29,
 7,
 0,
 11,
 4,
 2,
 0,
 3,
 2,
 3,
 6,
 2,
 5,
 5,
 37,
 2,
 8,
 3,
 6,
 2,
 7,
 3,
 6,
 28,
 3,
 3,
 7,
 4,
 3,
 1,
 5,
 2,
 62,
 10,
 4,
 8,
 21,
 4,
 2,
 5,
 1,
 1,
 2,
 5,
 1,
 5,
 2,
 4,
 8,
 132,
 13,
 8,
 7,
 14,
 12,
 15,
 13,
 50,
 2,
 7,
 7,
 10,
 2,
 6,
 15,
 1,
 724,
 30,
 6,
 4,
 3,
 1,
 9,
 2,
 3,
 2,
 46,
 4,
 4,
 2,
 13,
 2,
 8,
 3,
 10,
 36,
 1,
 5,
 4,
 3,
 3,
 3,
 7,
 10,
 71,
 10,
 6,
 13,
 2,
 24,
 0,
 0,
 4,
 3,
 0,
 2,
 7,
 8,
 8,
 5,
 3,
 16,
 254,
 1,
 3,
 1,
 22,
 0,
 0,
 0,
 0,
 1,
 7,
 3,
 11,
 2,
 15,
 5,
 205,
 1,
 40,
 0,
 2,
 16,
 2,
 2,
 0,
 7,
 11,
 11,
 120,
 42,
 3,
 2,
 0,
 2,
 2,
 14,
 8,
 11,
 31,
 4,
 2,
 1,
 2,
 13,
 2,
 6,
 1,
 2,
 3,
 19,
 15,
 1,
 7,
 7,
 20,
 2,
 4,
 168,
 15,
 38,
 1,
 0,
 3,
 11,
 2

In [451]:
kdtree['mass_cell'][:30][0]

[None,

In [435]:
root.childs

AttributeError: 'node' object has no attribute 'childs'

In [416]:
def tmpf(given, ilevel, ioct):
    mine = given%int( 10**(15-ilevel) )
    temp = mine//int(10**(15-ilevel-1))
    ncell = np.sum(temp == ioct)
    return ncell

In [411]:
octree['oct1']

array([111111100000000, 111111100000000, 111111100000000, ...,
       888888800000000, 888888800000000, 888888800000000])

In [417]:
np.sum([tmpf(octree['oct1'], 0, i+1) for i in range(8)])

11134418

In [418]:
inccell=0
def find_pos_lvl(ilevel,start,end, tmp=None):
    global kdtree, octree, inccell
    octids = octree['oct1']

    if(ilevel==0):
        kdtree['mass_cell'] = [0,0]
        ncell = np.sum([tmpf(octids, ilevel, i+1) for i in range(8)])
        kdtree['mass_cell'][0] = ncell
        kdtree['mass_cell'][1] = [[tmpf(octids, ilevel+1, i+1) for i in range(8)]]
    else:
        ncell = np.sum([tmpf(octids, ilevel, i+1) for i in range(8)])
        kdtree['mass_cell'][1].append([tmpf(octids, ilevel+1, i+1) for i in range(8)])

In [419]:
find_pos_lvl(0,0,0)

In [420]:
kdtree['mass_cell']

[11134418,
 [[240820, 247032, 4312975, 1092899, 259236, 269884, 4314428, 397144]]]

In [387]:
def searchlvl(givens, mylvl, myoct):
    global kdtree
    tmp = givens['oct1'] // int( 10**(15-mylvl) )
    ind = tmp == myoct
    mychild = givens['oct1'][ind] - myoct*int( 10**(15-mylvl) )
    for j in range(8):
        nchild, chchild = searchlvl(mychild, mylvl+1, j+1)
    return np.sum(ind), mychild

In [ ]:
def searchlvl(octarr, lvl):
    for i in range(8):
        tmp = octarr['oct1'] // 10**(15-lvl)


In [383]:
kdtree['mass_cell']

[11134418,
 [[4401030, [-1, -1, -1, -1, -1, -1, -1, -1]],
  [[4401030, [-1, -1, -1, -1, -1, -1, -1, -1]],
   [269003, [-1, -1, -1, -1, -1, -1, -1, -1]]],
  [240826, [-1, -1, -1, -1, -1, -1, -1, -1]],
  [264320, [-1, -1, -1, -1, -1, -1, -1, -1]],
  [5175286, [-1, -1, -1, -1, -1, -1, -1, -1]],
  [242939, [-1, -1, -1, -1, -1, -1, -1, -1]],
  [260857, [-1, -1, -1, -1, -1, -1, -1, -1]],
  [280157, [-1, -1, -1, -1, -1, -1, -1, -1]]]]

array([(111111120000000, 0, 8,  True), (111111150000000, 0, 8,  True),
       (111111160000000, 0, 8,  True), ...,
       (888888820000000, 0, 8,  True), (888888840000000, 0, 8,  True),
       (888888860000000, 0, 8,  True)],
      dtype=[('oct1', '<i8'), ('oct2', '<i8'), ('lvl', 'i1'), ('done', '?')])

In [327]:
np.sort(octarr['oct1'])

array([111111120000000, 111111150000000, 111111160000000, ...,
       888888820000000, 888888840000000, 888888860000000])

In [318]:
octarr[~octarr['done']]

array([(176838341453448,  430646668853801415, 36, False),
       (176838341453448,  430646668853801415, 36, False),
       (176838347612343, 1291940006561404245, 36, False), ...,
       (532483265443735, 1291940006561404245, 36, False),
       (532436825368454, 1722586675415205660, 36, False),
       (532436825368454, 1722586675415205660, 36, False)],
      dtype=[('oct1', '<i8'), ('oct2', '<i8'), ('lvl', 'i1'), ('done', '?')])

In [320]:
pos[~octarr['done']]

array([[-0.29177214, -0.12747123, -0.04635403],
       [-0.29177213, -0.12746964, -0.04635318],
       [-0.29168515, -0.12671173, -0.04394135],
       ...,
       [-0.26806529, -0.13870868,  0.03726277],
       [-0.28424719, -0.14704044,  0.02618831],
       [-0.28424521, -0.14703797,  0.02618508]])

In [321]:
cpos[~octarr['done']]

array([[-0.29177094, -0.12746429, -0.04634857],
       [-0.29177094, -0.12746429, -0.04634857],
       [-0.29167938, -0.12671661, -0.04393768],
       ...,
       [-0.26805878, -0.13871002,  0.03726959],
       [-0.28424835, -0.14704132,  0.02619171],
       [-0.28424835, -0.14704132,  0.02619171]], dtype=float128)

In [ ]:
# [lvl=0] 1 -> 8
# 11134418
# [lvl=1] 8 -> 64
# 11134418
# [lvl=2] 64 -> 512
# 11134418
# [lvl=3] 512 -> 4096
# 11134418
# [lvl=4] 4096 -> 32768
# 11134418
# [lvl=5] 32768 -> 255707
# 11097695
# [lvl=6] 255707 -> 928252
# 10505614
# [lvl=7] 928252 -> 1480365
# 9898697

In [286]:
octarr[['oct1', 'oct2']][0] == octarr[['oct1', 'oct2']]

array([ True, False, False, ..., False, False, False])

In [224]:
octarr

array([(111111111, 0, 9, False), (111111111, 0, 9, False),
       (111111111, 0, 9, False), ..., (222222222, 0, 9, False),
       (222222222, 0, 9, False), (222222222, 0, 9, False)],
      dtype=[('oct1', '<i8'), ('oct2', '<i8'), ('lvl', 'i1'), ('done', '?')])

In [181]:
octarr.dtype

dtype('<U30')

In [199]:
f"{np.float128(12345):.30f}"

'12345.000000000000000000000000000000'

In [204]:
f"{np.int64(999999999999999):30d}"

'               999999999999999'

In [169]:
np.float64(100000000000000000000)+np.float64(100000000000000000000)

2e+20

In [166]:
octarr = refine(pos, octarr, cpos)

[lvl=0] 1 -> 8
11134418
[lvl=1] 8 -> 64
11134418
[lvl=2] 64 -> 512
11134418
[lvl=3] 512 -> 4096
11134418
[lvl=4] 4096 -> 32768
11134418
[lvl=5] 32768 -> 255707
11097695
[lvl=6] 255707 -> 928252
10505614
[lvl=7] 928252 -> 1480365
9898697
[lvl=8] 1480365 -> 1931400
9424695
[lvl=9] 1931400 -> 2450569
8995940
[lvl=10] 2450569 -> 3385686
8392502
[lvl=11] 3385686 -> 5151826
7053424
[lvl=12] 5151826 -> 7528892
4776334
[lvl=13] 7528892 -> 9624522
2299613
[lvl=14] 9624522 -> 10695417
721405
[lvl=15] 10695417 -> 11021954
187342
[lvl=16] 11021954 -> 11105725
47397
[lvl=17] 11105725 -> 11126581
12778
[lvl=18] 11126581 -> 11128713
8833
[lvl=19] 11128713 -> 11128713
8833
[lvl=20] 11128713 -> 11128713
8833
[lvl=21] 11128713 -> 11127237
8833
[lvl=22] 11127237 -> 11127237
8833
[lvl=23] 11127237 -> 11127237
8833
[lvl=24] 11127237 -> 11126156
8833
[lvl=25] 11126156 -> 11126156
8833
[lvl=26] 11126156 -> 11126156
8833
[lvl=27] 11126156 -> 11125216
8833
[lvl=28] 11125216 -> 11125216
8833
[lvl=29] 11125216 -

KeyboardInterrupt: 

In [130]:
for _ in range(10):
    octarr, cpos = refine(pos, octarr, cpos)
    print(octarr[0], True in (octarr<0), cpos.shape)

[lvl=0] 1 -> 8
90.0 False (11134418, 3)
[lvl=1] 8 -> 64
900.0 False (11134418, 3)
[lvl=2] 64 -> 512
9000.0 False (11134418, 3)
[lvl=3] 512 -> 4096
90002.0 False (11134418, 3)
[lvl=4] 4096 -> 32768
900021.0 False (11134418, 3)
[lvl=5] 32768 -> 255707
9000212.0 True (11134418, 3)
[lvl=6] 255707 -> 928252
-90002124.0 True (11134418, 3)
[lvl=7] 928252 -> 1480365
-900021249.0 True (11134418, 3)
[lvl=8] 1480365 -> 1931400
-9000212499.0 True (11134418, 3)
[lvl=9] 1931400 -> 2450569
-90002124999.0 True (11134418, 3)


In [131]:
for _ in range(2):
    octarr, cpos = refine(pos, octarr, cpos)
    print(octarr[0], True in (octarr<0), cpos.shape)

[lvl=10] 2450569 -> 3385686
-900021249999.0 True (11134418, 3)
[lvl=11] 3385686 -> 5151826
-9000212499999.0 True (11134418, 3)


In [132]:
for _ in range(2):
    octarr, cpos = refine(pos, octarr, cpos)
    print(octarr[0], True in (octarr<0), cpos.shape)

[lvl=12] 5151826 -> 7528892
-90002124999999.0 True (11134418, 3)
[lvl=13] 7528892 -> 9624522
-900021249999999.0 True (11134418, 3)


In [133]:
for _ in range(2):
    octarr, cpos = refine(pos, octarr, cpos)
    print(octarr[0], True in (octarr<0), cpos.shape)

[lvl=14] 9624522 -> 10393025
-9000212499999999.0 True (11134418, 3)
[lvl=15] 10393025 -> 10393025
-9.0002125e+16 True (11134418, 3)


In [127]:
for _ in range(5):
    octarr, cpos = refine(pos, octarr, cpos)
    print(octarr[0], True in (octarr<0), cpos.shape)

[lvl=5] 32768 -> 255707
9000212.0 True (11134418, 3)
[lvl=6] 255707 -> 928252
-90002124.0 True (11134418, 3)
[lvl=7] 928252 -> 1480365
-900021240.0 True (11134418, 3)
[lvl=8] 1480365 -> 1931400
-9000212400.0 True (11134418, 3)
[lvl=9] 1931400 -> 2450569
-90002124000.0 True (11134418, 3)


In [123]:
octarr.shape

(11134418,)

In [117]:
pos

array([[-0.46253601, -0.41537952, -0.48716066],
       [-0.44583337, -0.40721586, -0.49342432],
       [-0.46430073, -0.4881639 , -0.409371  ],
       ...,
       [ 0.36647672, -0.13008   , -0.37999174],
       [ 0.36164023, -0.13857022, -0.38843454],
       [ 0.36129619, -0.12824863, -0.38781893]])

In [102]:
octarr,cpos = refine(pos, octarr, cpos)
octarr[0], True in (octarr<0), cpos.shape

[lvl=0] 1 -> 8


(90.0, False, (11134418, 3))

In [103]:
octarr,cpos = refine(pos, octarr, cpos)
octarr[0], True in (octarr<0), cpos.shape

[lvl=1] 8 -> 64


(900.0, False, (11134418, 3))

In [104]:
octarr,cpos = refine(pos, octarr, cpos)
octarr[0], True in (octarr<0), cpos.shape

[lvl=2] 64 -> 512


(9000.0, False, (11134418, 3))

In [105]:
octarr,cpos = refine(pos, octarr, cpos)
octarr[0], True in (octarr<0), cpos.shape

[lvl=3] 512 -> 4096


(90002.0, False, (11134418, 3))

In [106]:
octarr,cpos = refine(pos, octarr, cpos)
octarr[0], True in (octarr<0), cpos.shape

[lvl=4] 4096 -> 32768


(900021.0, False, (11134418, 3))

In [107]:
octarr,cpos = refine(pos, octarr, cpos)
octarr[0], True in (octarr<0), cpos.shape

[lvl=5] 32768 -> 255707


(9000212.0, True, (11134418, 3))

In [108]:
octarr,cpos = refine(pos, octarr, cpos)
octarr[0], True in (octarr<0), cpos.shape

[lvl=6] 255707 -> 928252


(-90002124.0, True, (11097695, 3))

In [109]:
octarr,cpos = refine(pos, octarr, cpos)
octarr[0], True in (octarr<0), cpos.shape

IndexError: boolean index did not match indexed array along dimension 0; dimension is 11097695 but corresponding boolean dimension is 11134418

In [99]:
octarr,cpos = refine(pos, octarr, cpos)
np.unique(octarr, return_counts=True), cpos[0], True in (octarr<0), cpos.shape

IndexError: boolean index did not match indexed array along dimension 0; dimension is 11097695 but corresponding boolean dimension is 11134418

In [74]:
octarr,cpos = refine(pos, octarr, cpos)
np.unique(octarr, return_counts=True), cpos[0]

((array([90., 91., 92., 93., 94., 95., 96., 97.]),
  array([4401030,  269003,  240826,  264320, 5175286,  242939,  260857,
          280157])),
 array([-0.25, -0.25, -0.25]))

In [75]:
octarr,cpos = refine(pos, octarr, cpos)
np.unique(octarr, return_counts=True), cpos[0]

((array([900., 901., 902., 903., 904., 905., 906., 907., 910., 911., 912.,
         913., 914., 915., 916., 917., 920., 921., 922., 923., 924., 925.,
         926., 927., 930., 931., 932., 933., 934., 935., 936., 937., 940.,
         941., 942., 943., 944., 945., 946., 947., 950., 951., 952., 953.,
         954., 955., 956., 957., 960., 961., 962., 963., 964., 965., 966.,
         967., 970., 971., 972., 973., 974., 975., 976., 977.]),
  array([  30283,   29256,   44892,   20525,   30450,   31992, 4043562,
          170070,   24633,   24289,   30542,   28530,   27185,   40538,
           53125,   40161,   32881,   26014,   31083,   26688,   37512,
           32139,   30724,   23785,   33333,   47635,   33017,   25276,
           22081,   45739,   33495,   23744,   31674,   31389, 4062565,
          884216,   46513,   33731,   50767,   34431,   21829,   25621,
           31802,   31876,   35320,   18272,   37623,   40596,   28122,
           22043,   39143,   39184,   29432,   37180,   

In [76]:
octarr,cpos = refine(pos, octarr, cpos)
np.unique(octarr, return_counts=True), cpos[0]

((array([9000., 9001., 9002., 9003., 9004., 9005., 9006., 9007., 9010.,
         9011., 9012., 9013., 9014., 9015., 9016., 9017., 9020., 9021.,
         9022., 9023., 9024., 9025., 9026., 9027., 9030., 9031., 9032.,
         9033., 9034., 9035., 9036., 9037., 9040., 9041., 9042., 9043.,
         9044., 9045., 9046., 9047., 9050., 9051., 9052., 9053., 9054.,
         9055., 9056., 9057., 9060., 9061., 9062., 9063., 9064., 9065.,
         9066., 9067., 9070., 9071., 9072., 9073., 9074., 9075., 9076.,
         9077., 9100., 9101., 9102., 9103., 9104., 9105., 9106., 9107.,
         9110., 9111., 9112., 9113., 9114., 9115., 9116., 9117., 9120.,
         9121., 9122., 9123., 9124., 9125., 9126., 9127., 9130., 9131.,
         9132., 9133., 9134., 9135., 9136., 9137., 9140., 9141., 9142.,
         9143., 9144., 9145., 9146., 9147., 9150., 9151., 9152., 9153.,
         9154., 9155., 9156., 9157., 9160., 9161., 9162., 9163., 9164.,
         9165., 9166., 9167., 9170., 9171., 9172., 9173., 9174.,

In [77]:
octarr,cpos = refine(pos, octarr, cpos)
np.unique(octarr, return_counts=True), cpos[0]

((array([90000., 90001., 90002., ..., 97775., 97776., 97777.]),
  array([ 510,  724, 1306, ...,  261, 1137,  734])),
 array([-0.46875, -0.40625, -0.46875]))

In [78]:
octarr,cpos = refine(pos, octarr, cpos)
np.unique(octarr, return_counts=True), cpos[0], True in octarr<0

((array([900000., 900001., 900002., ..., 977775., 977776., 977777.]),
  array([ 81,  31,  81, ...,  55,  74, 166])),
 array([-0.453125, -0.421875, -0.484375]),
 False)

In [79]:
octarr,cpos = refine(pos, octarr, cpos)
np.unique(octarr, return_counts=True), cpos[0], True in (octarr<0)

((array([-9777714., -9777677., -9777662., ...,  9777775.,  9777776.,
          9777777.]),
  array([ 1,  1,  1, ..., 21, 13, 24])),
 array([-0.4609375, -0.4140625, -0.4921875]),
 False)

In [80]:
octarr,cpos = refine(pos, octarr, cpos)
np.unique(octarr, return_counts=True), cpos[0], True in (octarr<0), cpos.shape

((array([-97777776., -97777767., -97777766., ...,  97777774.,  97777775.,
          97777777.]),
  array([1, 1, 1, ..., 8, 4, 3])),
 array([-0.46484375, -0.41796875, -0.48828125]),
 True)

In [83]:
octarr,cpos = refine(pos, octarr, cpos)
np.unique(octarr, return_counts=True), cpos[0], True in (octarr<0), cpos.shape

/tmp/ipykernel_17469/1562178021.py:3: RuntimeWarning: invalid value encountered in log10
  nlevel = int(np.log10(octarr[0]))


ValueError: cannot convert float NaN to integer

In [54]:
octarr,cpos = refine(pos, octarr, 0, cpos)
octarr

array([90., 90., 90., ..., 91., 91., 91.])

In [55]:
octarr,cpos = refine(pos, octarr, 0, cpos)
octarr

array([900., 900., 900., ..., 913., 913., 913.])

In [58]:
octarr,cpos = refine(pos, octarr, 0, cpos)
np.unique(octarr)

array([90070., 90161., 90252., 90343., 90434., 90525., 90616., 90707.,
       91070., 91161., 91252., 91343., 91434., 91525., 91616., 91707.,
       92070., 92161., 92252., 92343., 92434., 92525., 92616., 92707.,
       93070., 93161., 93252., 93343., 93434., 93525., 93616., 93707.,
       94070., 94161., 94252., 94343., 94434., 94525., 94616., 94707.,
       95070., 95161., 95252., 95343., 95434., 95525., 95616., 95707.,
       96070., 96161., 96252., 96343., 96434., 96525., 96616., 96707.,
       97070., 97161., 97252., 97343., 97434., 97525., 97616., 97707.])

In [59]:
octarr,cpos = refine(pos, octarr, 0, cpos)
np.unique(octarr)

array([900707., 901616., 902525., 903434., 904343., 905252., 906161.,
       907070., 910707., 911616., 912525., 913434., 914343., 915252.,
       916161., 917070., 920707., 921616., 922525., 923434., 924343.,
       925252., 926161., 927070., 930707., 931616., 932525., 933434.,
       934343., 935252., 936161., 937070., 940707., 941616., 942525.,
       943434., 944343., 945252., 946161., 947070., 950707., 951616.,
       952525., 953434., 954343., 955252., 956161., 957070., 960707.,
       961616., 962525., 963434., 964343., 965252., 966161., 967070.,
       970707., 971616., 972525., 973434., 974343., 975252., 976161.,
       977070.])

In [60]:
octarr,cpos = refine(pos, octarr, 0, cpos)
np.unique(octarr)

array([9007070., 9016161., 9025252., 9034343., 9043434., 9052525.,
       9061616., 9070707., 9107070., 9116161., 9125252., 9134343.,
       9143434., 9152525., 9161616., 9170707., 9207070., 9216161.,
       9225252., 9234343., 9243434., 9252525., 9261616., 9270707.,
       9307070., 9316161., 9325252., 9334343., 9343434., 9352525.,
       9361616., 9370707., 9407070., 9416161., 9425252., 9434343.,
       9443434., 9452525., 9461616., 9470707., 9507070., 9516161.,
       9525252., 9534343., 9543434., 9552525., 9561616., 9570707.,
       9607070., 9616161., 9625252., 9634343., 9643434., 9652525.,
       9661616., 9670707., 9707070., 9716161., 9725252., 9734343.,
       9743434., 9752525., 9761616., 9770707.])

In [61]:
octarr,cpos = refine(pos, octarr, 0, cpos)
np.unique(octarr)

array([90070707., 90161616., 90252525., 90343434., 90434343., 90525252.,
       90616161., 90707070., 91070707., 91161616., 91252525., 91343434.,
       91434343., 91525252., 91616161., 91707070., 92070707., 92161616.,
       92252525., 92343434., 92434343., 92525252., 92616161., 92707070.,
       93070707., 93161616., 93252525., 93343434., 93434343., 93525252.,
       93616161., 93707070., 94070707., 94161616., 94252525., 94343434.,
       94434343., 94525252., 94616161., 94707070., 95070707., 95161616.,
       95252525., 95343434., 95434343., 95525252., 95616161., 95707070.,
       96070707., 96161616., 96252525., 96343434., 96434343., 96525252.,
       96616161., 96707070., 97070707., 97161616., 97252525., 97343434.,
       97434343., 97525252., 97616161., 97707070.])

In [63]:
octarr,cpos = refine(pos, octarr, 0, cpos)
np.unique(octarr, return_counts=True)

(array([9.00707071e+09, 9.01616162e+09, 9.02525252e+09, 9.03434343e+09,
        9.04343434e+09, 9.05252525e+09, 9.06161616e+09, 9.07070707e+09,
        9.10707071e+09, 9.11616162e+09, 9.12525252e+09, 9.13434343e+09,
        9.14343434e+09, 9.15252525e+09, 9.16161616e+09, 9.17070707e+09,
        9.20707071e+09, 9.21616162e+09, 9.22525252e+09, 9.23434343e+09,
        9.24343434e+09, 9.25252525e+09, 9.26161616e+09, 9.27070707e+09,
        9.30707071e+09, 9.31616162e+09, 9.32525252e+09, 9.33434343e+09,
        9.34343434e+09, 9.35252525e+09, 9.36161616e+09, 9.37070707e+09,
        9.40707071e+09, 9.41616162e+09, 9.42525252e+09, 9.43434343e+09,
        9.44343434e+09, 9.45252525e+09, 9.46161616e+09, 9.47070707e+09,
        9.50707071e+09, 9.51616162e+09, 9.52525252e+09, 9.53434343e+09,
        9.54343434e+09, 9.55252525e+09, 9.56161616e+09, 9.57070707e+09,
        9.60707071e+09, 9.61616162e+09, 9.62525252e+09, 9.63434343e+09,
        9.64343434e+09, 9.65252525e+09, 9.66161616e+09, 9.670707

In [64]:
octarr,cpos = refine(pos, octarr, 0, cpos)
np.unique(octarr, return_counts=True)

(array([9.00707071e+10, 9.01616162e+10, 9.02525253e+10, 9.03434343e+10,
        9.04343434e+10, 9.05252525e+10, 9.06161616e+10, 9.07070707e+10,
        9.10707071e+10, 9.11616162e+10, 9.12525253e+10, 9.13434343e+10,
        9.14343434e+10, 9.15252525e+10, 9.16161616e+10, 9.17070707e+10,
        9.20707071e+10, 9.21616162e+10, 9.22525253e+10, 9.23434343e+10,
        9.24343434e+10, 9.25252525e+10, 9.26161616e+10, 9.27070707e+10,
        9.30707071e+10, 9.31616162e+10, 9.32525253e+10, 9.33434343e+10,
        9.34343434e+10, 9.35252525e+10, 9.36161616e+10, 9.37070707e+10,
        9.40707071e+10, 9.41616162e+10, 9.42525253e+10, 9.43434343e+10,
        9.44343434e+10, 9.45252525e+10, 9.46161616e+10, 9.47070707e+10,
        9.50707071e+10, 9.51616162e+10, 9.52525253e+10, 9.53434343e+10,
        9.54343434e+10, 9.55252525e+10, 9.56161616e+10, 9.57070707e+10,
        9.60707071e+10, 9.61616162e+10, 9.62525253e+10, 9.63434343e+10,
        9.64343434e+10, 9.65252525e+10, 9.66161616e+10, 9.670707

In [65]:
octarr,cpos = refine(pos, octarr, 0, cpos)
np.unique(octarr, return_counts=True)

(array([9.00707071e+11, 9.01616162e+11, 9.02525253e+11, 9.03434343e+11,
        9.04343434e+11, 9.05252525e+11, 9.06161616e+11, 9.07070707e+11,
        9.10707071e+11, 9.11616162e+11, 9.12525253e+11, 9.13434343e+11,
        9.14343434e+11, 9.15252525e+11, 9.16161616e+11, 9.17070707e+11,
        9.20707071e+11, 9.21616162e+11, 9.22525253e+11, 9.23434343e+11,
        9.24343434e+11, 9.25252525e+11, 9.26161616e+11, 9.27070707e+11,
        9.30707071e+11, 9.31616162e+11, 9.32525253e+11, 9.33434343e+11,
        9.34343434e+11, 9.35252525e+11, 9.36161616e+11, 9.37070707e+11,
        9.40707071e+11, 9.41616162e+11, 9.42525253e+11, 9.43434343e+11,
        9.44343434e+11, 9.45252525e+11, 9.46161616e+11, 9.47070707e+11,
        9.50707071e+11, 9.51616162e+11, 9.52525253e+11, 9.53434343e+11,
        9.54343434e+11, 9.55252525e+11, 9.56161616e+11, 9.57070707e+11,
        9.60707071e+11, 9.61616162e+11, 9.62525253e+11, 9.63434343e+11,
        9.64343434e+11, 9.65252525e+11, 9.66161616e+11, 9.670707

In [57]:
np.unique(octarr)

array([9007., 9016., 9025., 9034., 9043., 9052., 9061., 9070., 9107.,
       9116., 9125., 9134., 9143., 9152., 9161., 9170., 9207., 9216.,
       9225., 9234., 9243., 9252., 9261., 9270., 9307., 9316., 9325.,
       9334., 9343., 9352., 9361., 9370., 9407., 9416., 9425., 9434.,
       9443., 9452., 9461., 9470., 9507., 9516., 9525., 9534., 9543.,
       9552., 9561., 9570., 9607., 9616., 9625., 9634., 9643., 9652.,
       9661., 9670., 9707., 9716., 9725., 9734., 9743., 9752., 9761.,
       9770.])

In [68]:
cpos

array([[-0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25],
       ...,
       [ 0.25, -0.25, -0.25],
       [ 0.25, -0.25, -0.25],
       [ 0.25, -0.25, -0.25]])

In [ ]:
octarr, cpos

In [10]:
dm.shape

(11134418,)

In [33]:
octarr = pos2oct(dm['pos']-np.array([0.5,0.5,0.5]))

0 [0 0 0 0 0]
1 [0 0 0 0 0]
2 [0 0 0 0 0]
3 [2 2 4 6 2]
4 [1 1 1 0 1]
5 [2 3 4 3 7]


In [34]:
octarr

array([  138,   139,   268, ..., 45567, 45567, 45567])

In [35]:
oct(octarr[0])

'0o212'

In [62]:
arr = np.array(['0o00001', '0o00024'])
arr

array(['0o00001', '0o00024'], dtype='<U7')

In [77]:
arr = np.array(['0o00001', '0o00024'], dtype=oct)
arr

TypeError: Cannot interpret '<built-in function oct>' as a data type

In [85]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [84]:
8**30

1237940039285380274899124224

In [70]:
np.int_(arr)

ValueError: invalid literal for int() with base 10: '0o00001'

In [73]:
'0o00001'

'0o00001'

In [76]:
oct(52), type(oct(52))

('0o64', str)

In [72]:
np.int0(arr)

ValueError: invalid literal for int() with base 10: '0o00001'

In [68]:
from functools import partial
intoct = partial(int, base=8)
print(intoct('0o00001'))
arr.astype(intoct)

1


TypeError: Cannot interpret 'functools.partial(<class 'int'>, base=8)' as a data type

In [59]:
val = f"0o00001"
val

'0o00001'

In [60]:
int(val,8)

1

NameError: name 'temp' is not defined

In [54]:
np.log(919561)/np.log(8)

6.603528584956341

In [ ]:
val = f"0o{np.sum(a*temp):o}"
val

In [53]:
arrs = np.zeros(7, dtype='<U23')
for i in range(7):
    arr = np.random.randint(0,8,7)
    arrs[i] = arr2oct(arr)
arrs


919561
1950703
958946
1255438
1156953
1795803
870711


array(['0o3404011', '0o7341757', '0o3520742', '0o4624016', '0o4323531',
       '0o6663333', '0o3244467'], dtype='<U23')

In [35]:
len( arr2oct(a) )

8

In [36]:
val=arr2oct(a)
val

'0o123456'

In [46]:
int(val,8)

42798

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [41]:
int(val)

ValueError: invalid literal for int() with base 10: '0o123456'

In [32]:
ss = f"0o{a[0]}"
ss, 

TypeError: 'str' object cannot be interpreted as an integer

In [24]:
ref = time.time()
factorial(60)
print(time.time() - ref)

AssertionError: daemonic processes are not allowed to have children

In [5]:
import numpy as np

In [2]:
float(2866038.84679049) * float(2.78782)

7990000.417859464

In [3]:
float(2.78782)

2.78782

In [6]:
np.float32(2.78782)

2.78782

In [9]:
print(f"{np.float64(2.78782):.16f}")

2.7878200000000000


In [1]:
from rur import uri, uhmi
import numpy as np

In [2]:
xtest = np.array([-1,0,1])
arr = (xtest>=0).astype(int)
arr

array([0, 1, 1])

In [3]:
for i in range(0,1):
    print(i)

0


In [9]:
a = np.array([0,0,1,2,3,5,1,2,3,7,6,5,1,4,7,4,2,5,6,3,2,1])
ids = np.arange(len(a))+1
np.unique(a, return_counts=True,return_index=True)

(array([0, 1, 2, 3, 4, 5, 6, 7]),
 array([ 0,  2,  3,  4, 13,  5, 10,  9]),
 array([2, 4, 4, 3, 2, 3, 2, 2]))

In [23]:
np.double(-68.2484026551247)

-68.2484026551247

In [24]:
np.float64(-68.2484026551247)

-68.2484026551247

In [25]:
np.float32(-68.2484026551247)

-68.248405

In [26]:
type(0.0)

float

In [12]:
print(a)
print(ids)

[0 0 1 2 3 5 1 2 3 7 6 5 1 4 7 4 2 5 6 3 2 1]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]


In [22]:
arr = np.zeros(8)
for i in a:
    arr[i] += 1
arr

array([2., 4., 4., 3., 2., 3., 2., 2.])

In [19]:
arg = np.argsort(a, kind='mergesort')
arg

array([ 0,  1,  2,  6, 12, 21,  3,  7, 16, 20,  4,  8, 19, 13, 15,  5, 11,
       17, 10, 18,  9, 14])

In [13]:
arg = np.argsort(a, kind='stable')
arg

array([ 0,  1,  2,  6, 12, 21,  3,  7, 16, 20,  4,  8, 19, 13, 15,  5, 11,
       17, 10, 18,  9, 14])

In [8]:
a[arg]

array([0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 7, 7])

In [14]:
ids[arg]

array([ 1,  2,  3,  7, 13, 22,  4,  8, 17, 21,  5,  9, 20, 14, 16,  6, 12,
       18, 11, 19, 10, 15])

In [8]:
for i in range(10-1, 1-2,-1):
    print(i)

9
8
7
6
5
4
3
2
1
0


In [17]:
repo = '/storage5/FORNAX/KISTI_OUTPUT/l10006'
snap = uri.RamsesSnapshot(repo, iout=150, mode='fornax')
hals = uhmi.HaloMaker.load(snap, galaxy=False)
np.sum(hals['mvir'] > 10**13)

[Output 00150] Age (Gyr) : 4.310 / 13.773, z = 1.48304 (a = 0.4027)


42

In [16]:
repo = '/storage7/NewCluster'
snap = uri.RamsesSnapshot(repo, iout=-1, mode='nc')
hals = uhmi.HaloMaker.load(snap, galaxy=False)
np.sum(hals['mvir'] > 10**13)

[Output 00299] Age (Gyr) : 4.406 / 13.761, z = 1.48701 (a = 0.4021)


35

In [2]:
from collections import defaultdict
import time
import numpy as np
from multiprocessing import shared_memory

mem = defaultdict(lambda: None)
mem_address = defaultdict(lambda: None)

def allocate(name, shape, dtype=np.float64):
    global mem
    print(f"Allocate `{name}`");time.sleep(1)
    arr = np.empty(shape, dtype=dtype)
    temp = shared_memory.SharedMemory(create=True, size=arr.nbytes, name=name)
    assert mem_address[name] is None
    mem_address[name] = temp
    mem[name] = np.ndarray(arr.shape, dtype=dtype, buffer=mem_address[name].buf)
    arr = None

def deallocate(*names):
    for name in names:
        print(f"Deallocate `{name}`");time.sleep(1)
        if mem_address[name] is not None:
            mem[name] = None
            mem_address[name].shm.close()
            mem_address[name].shm.unlink()
            mem_address[name] = None

In [8]:
a = np.random.rand(10,10,10)
i,j,k = np.where(a == a.max())
i,j,k

(array([9]), array([2]), array([2]))

In [1]:
from halo_defs import frange
for i in frange(10,1,-1):
    print(i)

10
9
8
7
6
5
4
3
2
1


In [16]:
    f20 = open(f'input_HaloMaker.dat', 'r')
    for line in f20:
        i = line.find('=')
        if (i == -1 or line[0] == '#'): continue
        name  = line[:i].strip()
        value = line[i+1:].strip()
        # check for a comment at end of line
        i     = value.find('!')
        if (i != -1): value = value[:i].strip()
        print(f'> {name} : {value}')
        if (name == 'omega_0' or name == 'Omega_0' or name == 'omega_f'):
            omega_f = float(value)
        elif (name == 'omega_l' or name == 'lambda_0' or name == 'lambda_f'):
            omega_lambda_f = float(value)
        elif (name == 'af' or name == 'afinal' or name == 'a_f'):
            af = float(value)
        elif (name == 'Lf' or name == 'lf' or name == 'lbox'):
            Lf = float(value)
        elif (name == 'H_f' or name == 'H_0' or name == 'H'):
            H_f = float(value)
        elif (name == 'FlagPeriod'):
            FlagPeriod = int(value)
        elif (name == 'n' or name == 'N' or name == 'npart'):
            nMembers = int(value)
        elif (name == 'cdm'):
            cdm = value=='.true.'
        elif (name == 'method'):
            method = value
        elif (name == 'b'):
            b_init = float(value)
        elif (name == 'nvoisins'):
            nvoisins = int(value)
        elif (name == 'nhop'):
            nhop = int(value)
        elif (name == 'rhot'):
            rho_threshold = float(value)
        elif (name == 'fudge'):
            fudge = float(value)
        elif (name == 'fudgepsilon'):
            fudgepsilon = float(value)
        elif (name == 'alphap'):
            alphap = float(value)
        elif (name == 'verbose'):
            verbose = value=='.true.'
        elif (name == 'megaverbose'):
            megaverbose = value=='.true.'
        elif (name == 'DPMMC'):
            DPMMC = value=='.true.'
        elif (name == 'SC'):
            SC = value=='.true.'
        elif (name == 'dcell_min'):
            dcell_min = float(value)
        elif (name == 'eps_SC'):
            eps_SC = float(value)
        elif (name == 'nsteps' or name == 'nsteps_do'):
            nsteps = int(value)
        elif (name == 'dump_DMs'):
            dump_dms = value=='.true.'; print(dump_dms)
        elif (name == 'agor_file'):
            if(ANG_MOM_OF_R): agor_file = f"{data_dir}/{value}"
        else:
            print(f'>   dont recognise parameter: {name}')
    f20.close()

> af : 1.000
> lbox : 142.04546
> H_f : 70.400
> omega_f : 0.272
> lambda_f : 0.728
> npart : 100
> method : MSM
> cdm : .false.
> b : 0.02
> nvoisins : 20
> nhop : 20
> rhot : 178.0
> fudge : 5.
> fudgepsilon : 0.00002
> alphap : 2.5
> verbose : .false.
> megaverbose : .false.
> nsteps : 1026
> FlagPeriod : 1
> dump_stars : .true.
>   dont recognise parameter: dump_stars
> nchem : 0
>   dont recognise parameter: nchem
> SC : .true.
> dcell_min : 4e-05
> eps_SC : 0.3


In [11]:
bool('.false.')

True

In [10]:
a = {'a': 1, 'b': 2, 'c': 3}
len(a.keys())

3

In [2]:
allocate("a", (1000, 1000))

Allocate `a`


In [8]:
mem_address['a'].unlink()

In [9]:
mem['a']

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [1]:
import numpy as np

In [2]:
import input_output

In [3]:
input_output.read_data()


 > In read_data: timestep  ---> 1
> data_dir: `.`
> name_of_file: `/storage7/NH2/snapshots/output_00760/`
 > From AMR file: `/storage7/NH2/snapshots/output_00760//info_00760.txt`
 	ncpu=480, ndim=3, nstep=41323
 	nx=1, ny=1, nz=1
 	nlevelmax=21, ngridmax=3000000
 	t=-2.238e-01, aexp=0.000e+00, hexp=7.040e+01
 	omega_m=0.272, omega_l=0.728, omega_k=0.000, omega_b=0.045
 	boxlen=3.609e+26 h-1 Mpc
> Found 198022204 non-stellar particles
> Found 106893518 star particles
> Reading positions and masses...
Allocate `pos_tmp`
Allocate `vel_tmp`
Allocate `mass_tmp`


Reading particles:   0%|          | 0/480 [00:00<?, ?cpu/s]

Allocate `tmpp`


: 

In [ ]:
repository = f"/storage7/NewCluster/snapshots/output_00001/"
# Find the index where the `output_` string starts
index = repository.find("output_")
index

31

In [ ]:
var = {}

In [ ]:
from collections import defaultdict
var = defaultdict(lambda: None)

var[0]

In [ ]:
from rur.fortranfile import FortranFile
import numpy as np

In [ ]:
fname = "/storage7/NH2/snapshots/output_00811/amr_00811.out00001"
with FortranFile(fname, 'r') as f:
    ncpu, = f.read_ints()
    ndim, = f.read_ints()
    nx,ny,nz = f.read_ints()
    nlevelmax, = f.read_ints()
    ngridmax, = f.read_ints()
    idum, = f.read_ints()
    idum, = f.read_ints()
    boxlen, = f.read_reals()
    nout,idum,idum = f.read_ints()
    idum = f.read_reals()
    idum = f.read_reals()
    tco, = f.read_reals()
    idum = f.read_reals()
    idum = f.read_reals()
    idum, nstep_coarse = f.read_ints()
    idum = f.read_reals(); print(idum)
    print(f.read_reals())
    print(f.read_reals())
    print(f.read_reals())

[-6.26297462e-09  1.67279403e-01  9.99999829e-01]
[2.72000015e-01 7.27999985e-01 0.00000000e+00 4.55000000e-02
 7.04000015e+01 2.19320115e-02 1.00000003e+02]
[ 8.63772048e-01  8.00127613e-01  8.63731715e-01 -3.46636006e-04
 -9.32188145e-04]
[2.43423569e-12]


In [ ]:
fname = "/storage6/NewHorizon/snapshots/output_01026/amr_01026.out00001"
with FortranFile(fname, 'r') as f:
    ncpu, = f.read_ints()
    ndim, = f.read_ints()
    nx,ny,nz = f.read_ints()
    nlevelmax, = f.read_ints()
    ngridmax, = f.read_ints()
    idum, = f.read_ints()
    idum, = f.read_ints()
    boxlen, = f.read_reals()
    nout,idum,idum = f.read_ints()
    idum = f.read_reals()
    idum = f.read_reals()
    tco, = f.read_reals()
    idum = f.read_reals()
    idum = f.read_reals()
    idum, nstep_coarse = f.read_ints()
    idum = f.read_reals(); print(idum)
    print(f.read_reals())
    print(f.read_reals())
    print(f.read_reals())

[-6.28479468e-09  1.67279403e-01  9.99999998e-01]
[2.72000015e-01 7.27999985e-01 0.00000000e+00 4.55000000e-02
 7.04000015e+01 2.19320115e-02 1.00000003e+02]
[ 8.53613930e-01  7.86582186e-01  8.53572106e-01 -3.59677480e-04
 -9.90379068e-04]
[2.43423569e-12]


In [ ]:
fname = "/storage6/NewHorizon/snapshots/output_00001/part_00001.out00001"
with FortranFile(fname, 'r') as f:
    ncpu, = f.read_ints()
    ndim, = f.read_ints()
    nx,ny,nz = f.read_ints()
    nlevelmax, = f.read_ints()
    ngridmax, = f.read_ints()
    idum, = f.read_ints()
    idum, = f.read_ints()
    boxlen, = f.read_reals()
    nout,idum,idum = f.read_ints()
    idum = f.read_reals()
    idum = f.read_reals()
    tco, = f.read_reals()
    idum = f.read_reals()
    idum = f.read_reals()
    idum, nstep_coarse = f.read_ints()
    idum = f.read_reals(); print(idum)
    print(f.read_reals())
    print(f.read_reals())
    print(f.read_reals())


ValueError: not enough values to unpack (expected 3, got 1)

In [ ]:
var['allocated'] = np.array([])


In [ ]:
def func():
    var['allocated'] = np.empty((1,3))

func()

In [ ]:
print(var)

{'allocated': array([[4.66809194e-310, 0.00000000e+000, 1.58101007e-322]])}


In [ ]:
import numpy as np

def polint(xa, ya, x):
    n = len(xa)
    if n > 10:
        raise ValueError("Maximum allowed size for xa and ya is 10")
    
    c = ya.copy()
    d = ya.copy()

    # Find the index of the closest value in xa to x
    ns = np.abs(xa - x).argmin()
    y = ya[ns]
    
    for m in range(1, n):
        den = xa[:n-m] - x - xa[m:] + x
        w = c[1:n-m+1] - d[:n-m]
        with np.errstate(divide='raise'):
            try:
                den = w / den
            except FloatingPointError:
                raise ValueError("Denominator is zero!")

        d[:n-m] = (xa[m:] - x) * den
        c[:n-m] = (xa[:n-m] - x) * den

        if 2 * ns < n - m:
            dy = c[ns]
        else:
            dy = d[ns-1]
            ns -= 1
        
        y += dy
        
    return y, dy


In [ ]:
def qromo(func, a, b, **kwargs):
    def midpnt(func, a, b, s, n, **kwargs):
        # If `it` was a static or saved variable in Fortran subroutine,
        # then in Python, it can be made an attribute of the function to mimic this behavior.
        if not hasattr(midpnt, "it"):
            midpnt.it = 0

        if n == 1:
            s = (b - a) * func(0.5 * (a + b), **kwargs)
            midpnt.it = 1
        else:
            tnm = midpnt.it
            del_ = (b - a) / (3.0 * tnm)  # using del_ instead of del because del is a keyword in Python
            ddel = del_ + del_
            x = a + 0.5 * del_
            sum_ = 0.0  # using sum_ instead of sum because sum is a built-in function in Python

            for j in range(midpnt.it):
                sum_ += func(x, **kwargs)
                x += ddel
                sum_ += func(x, **kwargs)
                x += del_
            
            s = (s + (b - a) * sum_ / tnm) / 3.0
            midpnt.it *= 3

        return s

    jmax = 14; k = 5; km = k - 1; jmaxp = jmax + 1; dss=0
    eps = 1e-6
    s = np.zeros(jmaxp)
    h = np.zeros(jmaxp)
    h[0] = 1.0
    for j in range(jmax):
        s[j] = midpnt(func, a, b, s[j], j+1, **kwargs)
        if j >= k:
            ss, dss = polint(h[j-km : j-km+k], s[j-km : j-km+k], 0.0)
            if abs(dss) < eps * abs(ss):
                return ss
        s[j+1] = s[j]
        h[j+1] = h[j] / 9.0
    print('> too many steps in qromo')
    return ss

In [ ]:
# def midpnt(func, a, b, s, n, **kwargs):
#     # If `it` was a static or saved variable in Fortran subroutine,
#     # then in Python, it can be made an attribute of the function to mimic this behavior.
#     if not hasattr(midpnt, "it"):
#         midpnt.it = 0

#     if n == 1:
#         s = (b - a) * func(0.5 * (a + b), **kwargs)
#         midpnt.it = 1
#     else:
#         tnm = midpnt.it
#         del_ = (b - a) / (3.0 * tnm)  # using del_ instead of del because del is a keyword in Python
#         ddel = del_ + del_
#         x = a + 0.5 * del_
#         sum_ = 0.0  # using sum_ instead of sum because sum is a built-in function in Python

#         for j in range(midpnt.it):
#             sum_ += func(x, **kwargs)
#             x += ddel
#             sum_ += func(x, **kwargs)
#             x += del_
        
#         s = (s + (b - a) * sum_ / tnm) / 3.0
#         midpnt.it *= 3

#     return s


In [ ]:
def age_temps(x,omm=0,oml=0):
    return 1/np.sqrt(omm*x**5 + (1.0-omm-oml)*x**4 + oml*x**2)

def age_temps_turn_around(x,omm=0,oml=0):
    temps = omm*x**5 + (-omm-oml)*x**4 + oml*x**2
    if(temps > 0.0):
       return 1/np.sqrt(temps)
    else:
       return 0.0

In [ ]:
somme0 = qromo(age_temps, 1, 10001, omm=0.272, oml=0.728)
print(somme0)
print(977.78*somme0/70.4)

0.9906541773545013
13.759117067239833


In [ ]:
a = qromo(age_temps_turn_around, 101, 10001, omm=0.272, oml=0.728)
b = qromo(age_temps_turn_around, 1, 101, omm=0.272, oml=0.728)
print(a)
print(b)
print(a+b)

0.0012719822478692107
0.4751694125467955
0.47644139479466474


In [ ]:
qromo(age_temps_turn_around, 1, 10001, omm=0.272, oml=0.728)

KeyboardInterrupt: 

In [ ]:
import numpy as np

def cubic(a1, a2, a3, a4):
    """
    Function for evaluating a cubic equation. In the case where
    there is one real root, this routine returns it. If there are three
    roots, it returns the smallest positive one.

    The cubic equation is a1*x**3 + a2*x**2 + a3*x + a4 = 0.

    Ref: Tallarida, R. J., "Pocket Book of Integrals and Mathematical
    Formulas," 2nd ed., Boca Raton: CRC Press 1992, pp. 8-9.
    """

    if a1 == 0.0:
        raise ValueError('Quadratic/linear equation passed to cubic function')

    p = a3/a1 - (a2/a1)**2/3.0
    q = a4/a1 - a2*a3/a1**2/3.0 + 2.0*(a2/a1)**3/27.0
    d = p**3/27.0 + q**2/4.0

    if d > 0.0:
        a = -0.5*q + np.sqrt(d)
        b = -0.5*q - np.sqrt(d)
        a = a**(1/3) if a > 0 else -(-a)**(1/3)
        b = b**(1/3) if b > 0 else -(-b)**(1/3)
        y = a + b
    else:
        ar = -0.5 * q
        ai = np.sqrt(-d)
        r = (ar**2 + ai**2)**(1/6)
        theta = np.arctan2(ai, ar)
        y1 = 2.0 * r * np.cos(theta/3.0) - a2/a1/3.0
        y = y1
        y2 = 2.0 * r * np.cos(theta/3.0 + 2.0*np.pi/3.0) - a2/a1/3.0
        if y < 0 or (y2 > 0 and y2 < y): y = y2
        y3 = 2.0 * r * np.cos(theta/3.0 - 2.0*np.pi/3.0) - a2/a1/3.0
        if y < 0 or (y3 > 0 and y3 < y): y = y3

    return y

# Example
print(cubic(1, -6, 11, -6))  # It should return 3.0 which is the smallest positive root for x^3 - 6x^2 + 11x - 6 = 0

1.0
